# Otras funcionalidades de Julia

### Funciones Mutables vs No Mutables 

Por convención, funciones seguidas de `!` alteran sus contenidos y funciones que no lo tienen no pueden ser modificadas.

In [ ]:
v = [3, 5, 2]

In [ ]:
sort(v)

In [ ]:
v

`sort(v)` devuelve un arrego ordenado que contiene los mismos elementos de `v` sin alterar a `v`.


In [ ]:
sort!(v)

In [ ]:
v

`sort!(v)` altera el argumento `v` que se le pasa a la función.

## Funciones de orden superior 

La función `map()` es una función en Julia que toma como uno de sus argumentos de entrada una _función_. `map()` aplica la función a cada elemento de la estructura de datos que se le pasa.

Por ejemplo, al ejecutar:
```julia
map(f,[1, 2, 3])
```
obtendremos un arreglo de salida donde la función `f()` se aplica a cada uno de los elementos del array `[1, 2, 3]`, es decir,
```julia
[f(1), f(2), f(3)]
```

Se han elevado al cuadrado todos los elementos de `[1, 2, 3]` en lugar de elevar al cuadrado el vector mismo. Para hacer esto, se puede pasar a la función `map()` una función anónima en lugar del nombre de la función: 

In [ ]:
g = x -> x^2
g(5)

In [ ]:
map(g, [1,2,3])

In [ ]:
f = x -> x^3

In [ ]:
map(x -> x^3, [1, 2, 3])

### Broadcast 

`broadcast` es otra función de orden superior como `map`. `broadcast` es una generalización de `map`, por lo que puede hacer cualquier cosa que hace `map` e incluso algo más. La sintáxis de `broadcast` es la misma que la de `map`.

In [ ]:
broadcast(f, [1, 2, 3])

Una vez más se ha aplicado `f` a todos los elementos de `[1, 2, 3]`, esta vez difundiendo (_broadcasting_) la función `f`.
Un poco de "syntactic sugar" para llamar a `broadcast` es colocar un `.` entre el nombre de la función que se desee hacer `brodcast` y sus argumentos de entrada, por ejemplo:
```julia
broadcast(f,[1, 2, 3])
```
es lo mismo que escribir,
```julia
f.([1, 2, 3])
```

In [ ]:
f.([1, 2, 3])

In [ ]:
@time broadcast(f,[1, 2, 3])

In [ ]:
@time f.([1, 2, 3])

Notemos la diferencia entre hacer la llamada a la función:
```julia 
f([1, 2, 3])
```
y hacer el _broadcast_ de la función.

Es posible elevar al cuadrado cada elemento del vector, pero no es posible elevar al cuadrado al vector.

In [ ]:
f([1, 2, 3])

Entiéndase la operación de **Broadcast** como una "expansión la dimensión unaria" de los objetos que se pasan a la función.

Para hacerlo más claro, apliquemos `f()` a una matriz `A` y veamos la diferencia al hacer un _broadcast_ a `f()` sobre `A`

In [ ]:
A = [i + 3*j for j in 0:2, i in 1:3]

In [ ]:
f(A)  # multiplica A por si misma 3 veces

In [ ]:
f.(A)

Con el _broadcast_ cada elemento de `A` se elevan al cuadrado.

La notación con `.` para hacer el _broadcast_ permite escribir composiciones complejas de expresiones en una manera que parece más natural y cercana a la notación matemática convencional.

In [ ]:
A

Podemos escribir

In [ ]:
A .+ 2 .* f.(A) ./ A   

que será mucho más natural que escribir:

In [ ]:
broadcast(x -> x + 2 * f(x) / x, A)

**Ejemplo:**

Evaluación de varias funciones sobre un arreglo de números aleatorios

In [101]:
funs = [sin,cos]

2-element Array{Function,1}:
 sin (generic function with 13 methods)
 cos (generic function with 13 methods)

In [102]:
vals = rand(5)

5-element Array{Float64,1}:
 0.9223278736331983
 0.7525848792803029
 0.9294946140728197
 0.3471880980420521
 0.7440681193198235

In [103]:
[[f(v) for f in funs] for v in vals]

5-element Array{Array{Float64,1},1}:
 [0.7970097358518801, 0.6039664568147112]
 [0.6835278080961265, 0.7299244725033576]
 [0.8013177016423502, 0.5982390333592597]
 [0.34025503135578317, 0.9403331928827542]
 [0.6772865019645169, 0.7357193719460352]

In [106]:
map.(funs, Ref(vals))

2-element Array{Array{Float64,1},1}:
 [0.7970097358518801, 0.6835278080961265, 0.8013177016423502, 0.34025503135578317, 0.6772865019645169]
 [0.6039664568147112, 0.7299244725033576, 0.5982390333592597, 0.9403331928827542, 0.7357193719460352]

In [107]:
opera(f, x) = f(x)
opera.(funs, permutedims(vals))

2×5 Array{Float64,2}:
 0.79701   0.683528  0.801318  0.340255  0.677287
 0.603966  0.729924  0.598239  0.940333  0.735719

## Interoperabilidad

### Interoperabilidad con Python: `PyCall`

Tomando en cuenta el ecosistema de paquetes de Python, la gama de paquetes disponibles en Julia puede parecer un tanto limitada. Sin embargo, esto se compensa con la facilidad de llamar a paquetes escritos en otros idiomas desde Julia.

In [ ]:
using Pkg
Pkg.add("PyCall")

using PyCall

`PyCall` tiene una interfase de alto nivel diseñada para el transporte de código entre Julia y Python y es transparente desde el punto de vista del usuario. Por ejemplo, para importar el mótudlo `math` de Python utilizamos el método `pyimport()`:

In [108]:
pymath = pyimport("math")

PyObject <module 'math' from '/home/oscar/anaconda3/lib/python3.6/lib-dynload/math.cpython-36m-x86_64-linux-gnu.so'>

In [109]:
pymath.sin(1)

0.8414709848078965

In [110]:
pymath.sin(0.3 * pymath.pi) - sin(0.3 * pymath.pi)

0.0

In [111]:
nprandom = pyimport("numpy.random")

PyObject <module 'numpy.random' from '/home/oscar/anaconda3/lib/python3.6/site-packages/numpy/random/__init__.py'>

In [112]:
nprandom.rand(3,4)

3×4 Array{Float64,2}:
 0.39595   0.804066  0.585287  0.612368
 0.33574   0.132703  0.152141  0.21491
 0.284904  0.671616  0.654112  0.00574296

Al hacer una llamada a `arrays` de  `numpy` estos son convertidos automáticamente a arreglos de Julia.

In [113]:
jarray = nprandom.rand(3,4)
typeof(jarray)

Array{Float64,2}

Definamos una función:

In [114]:
objective = x -> cos(x) - x

#47 (generic function with 1 method)

In [115]:
objective(3)

-3.989992496600445

Podemos pasar esta función de Julia a un módulo de Python:

In [116]:
so = pyimport("scipy.optimize")
so.newton(objective, 1)

0.7390851332151607

Adicionalmente, podemos consultar la ayuda de la función `newton()` en Python:

In [117]:
?so.newton


    Find a zero using the Newton-Raphson or secant method.

    Find a zero of the function `func` given a nearby starting point `x0`.
    The Newton-Raphson method is used if the derivative `fprime` of `func`
    is provided, otherwise the secant method is used.  If the second order
    derivative `fprime2` of `func` is provided, then Halley's method is used.

    Parameters
    ----------
    func : function
        The function whose zero is wanted. It must be a function of a
        single variable of the form f(x,a,b,c...), where a,b,c... are extra
        arguments that can be passed in the `args` parameter.
    x0 : float
        An initial estimate of the zero that should be somewhere near the
        actual zero.
    fprime : function, optional
        The derivative of the function when available and convenient. If it
        is None (default), then the secant method is used.
    args : tuple, optional
        Extra arguments to be used in the function call.
    tol : float,

In [91]:
integrate = pyimport("scipy.integrate")

PyObject <module 'scipy.integrate' from '/home/oscar/anaconda3/lib/python3.6/site-packages/scipy/integrate/__init__.py'>

In [98]:
f1(x,t) = -x

f1 (generic function with 1 method)

In [96]:
t = 0:0.1:10

0.0:0.1:10.0

In [100]:
soln = integrate.odeint(f1, 1, t);